# Make EGFR Figure 2

This notebooks takes all the trans genes that have positive and negative results and runs a GSEA using Reactome. It then takes a subset of genes from the top hit(Hemostasis) pathway and maps them on a large circle heat map. This heatmap focuses on coagulation and urokinase related genes. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas.util.testing as tm
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest

import gseapy as gp
from gseapy.plot import barplot, dotplot

import cptac
import cptac.utils as u
import plot_utils as p 

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  """


# Step 1: Find Trans proteins with opposite effects in different cancers 

Load df with all of the genes that are FDR significant.

In [2]:
FDR_sig = pd.read_csv("Pval_corr_table_Fig_2_return_sig.csv")
FDR_sig = FDR_sig.replace(to_replace ='_proteomics', value = '', regex = True) #Remove proteomics labeling
FDR_sig= FDR_sig.drop(['Unnamed: 0'], axis=1)
FDR_sig = FDR_sig.set_index("Comparison")
FDR_sig = FDR_sig[1:] #Drop EGFR 
FDR_sig

,Correlation_Gbm,P_value_Gbm,Correlation_kidney,P_value_kidney,Correlation_Ovar,P_value_Ovar,Correlation_Brca,P_value_Brca,Correlation_Luad,P_value_Luad,Correlation_hnscc,P_value_hnscc,Correlation_Lscc,P_value_Lscc,Correlation_Colon,P_value_Colon
Comparison,,,,,,,,,,,,,,,,
PHLDA1,0.816848,3.507071e-21,NaN,NaN,NaN,NaN,0.364797,0.002164,NaN,NaN,0.664271,8.888640e-12,0.713420,2.644826e-14,NaN,NaN
GRB2,-0.610889,6.729990e-08,NaN,NaN,NaN,NaN,NaN,NaN,-0.302439,0.020631,-0.532341,3.320092e-06,NaN,NaN,NaN,NaN
CDH4,0.559180,3.420388e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PLA2G15,-0.556624,3.420388e-06,-0.298029,0.02216,NaN,NaN,0.274185,0.016061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOCS2,0.562720,3.420388e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.472624,1.417921e-02,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCDC6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.287735,0.049589
SSR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.301814,0.049589
PSMB1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.287839,0.049589


# Step 3 Make Data frame for figure 2

In [10]:
# Sig PTEN # CD2? CHUK
t_only = ['CD3E', 'CD3G', 'CD4', 'CD8A', 'GRAP2', 'FYB1', 'PRKCQ'] 
b_only = ['PPP3CC', 'CD79A', 'SYK', 'BTK', 'LYN', 'BLNK', 'BLK', 'HLA-DMA', 'HLA-DMB']
both = ['CD2', 'CD5', 'GRB2', 'LCK', 'ZAP70', 'VAV1', 'PTPN6', 'PTPRC', 'PIK3CD', 'PIK3CG', 'INPP5D',
        'PLCG1', 'PLCG2', 'NFATC2', 'PRKCB', 'CHUK', 'NFKB2', 'DOCK2', 'RAC2', 'ELMO1', 'WAS']
immune = t_only + b_only + both

In [11]:
#Get append version of the df with all cancer type, fdr sig trans results
df_FDR_append = pd.read_csv("Pval_corr_table_Fig_2_return_sig_append.csv")
df_FDR_append = df_FDR_append.drop(['Unnamed: 0'], axis=1)

#Remove Proteomics labeling 
df_FDR_append = df_FDR_append.replace(to_replace ='_proteomics', value = '', regex = True) 

#subset dataframe to include genes only desired for figure 
df_FDR_append= df_FDR_append[df_FDR_append.Comparison.isin(immune)]
df_FDR_append['Cancer Type'].unique()
df_FDR_append

,Comparison,Correlation,P_value,Cancer Type
1,GRB2,-0.610889,6.729990e-08,GBM
24,WAS,-0.501918,4.961122e-05,GBM
72,PLCG2,-0.472453,1.145235e-04,GBM
75,SYK,-0.470849,1.202510e-04,GBM
82,CD4,-0.467479,1.362923e-04,GBM
...,...,...,...,...
7781,PRKCQ,-0.284462,3.293176e-02,Head and Neck
7876,RAC2,-0.277100,3.855935e-02,Head and Neck
8461,NFATC2,-0.297428,4.438700e-02,Lscc
8573,PRKCB,0.395927,6.516094e-03,Colon


Set add new column to be unique index and order the new index. This way genes will be grouped by coagulation factors, regulators, and urokinase genes.

In [ ]:
df_FDR_append["Index"] = df_FDR_append["Comparison"] + " " + df_FDR_append["Cancer Type"]
df_FDR_append = df_FDR_append.set_index("Index")


ordered_list = []
cancer_list = ['Breast', 'Colon', 'GBM', 'Head and Neck', 'Kidney', 'Lscc', 'Luad']
for gene in immune:
    for cancer in cancer_list:
        ordered_list.append(gene+' '+cancer)

ordered_df = df_FDR_append.reindex(ordered_list)
ordered_df = ordered_df.loc[ordered_df['P_value'] < 0.05] # Keep sig 
     

In [18]:
                               
ordered_df = ordered_df.replace({'Head and Neck': 'HNSCC', 'Luad': 'LUAD', 'Lscc': 'LSCC', 'Ovarian': 'OV', 'Colon': 'CO',
                    'Kidney': 'ccRCC', 'Breast': 'BR', 'Gbm': 'GBM', 'Endo': 'EC'})
ordered_df['Cancer Type'].unique()

array(['HNSCC', 'LUAD', 'GBM', 'ccRCC', 'BR', 'LSCC', 'CO'], dtype=object)

# Step 4: Plot Figure 2

In [20]:
legend_min = ordered_df["P_value"].min()
#Make plot using plot utils
p.plotCircleHeatMap(ordered_df, circle_var = "P_value",color_var = "Correlation", x_axis = "Comparison", 
                    y_axis = "Cancer Type", plot_width= 1000, plot_height = 500, legend_min = 1e-8, 
                    legend_med = 0.00001, legend_max = 0.05, font_size = 12, show_legend = True , save_png = "Figure2.png")

Loading BokehJS ...

In [ ]:
# Sig PTEN # CD2? CHUK
t_only = ['CD3E', 'CD3G', 'CD4', 'CD8A', 'GRAP2', 'FYB1', 'PRKCQ'] 
b_only = ['PPP3CC', 'CD79A', 'SYK', 'BTK', 'LYN', 'BLNK', 'BLK', 'HLA-DMA', 'HLA-DMB']
both = ['CD2', 'CD5', 'GRB2', 'LCK', 'ZAP70', 'VAV1', 'PTPN6', 'PTPRC', 'PIK3CD', 'PIK3CG', 'INPP5D',
        'PLCG1', 'PLCG2', 'NFATC2', 'PRKCB', 'CHUK', 'NFKB2', 'DOCK2', 'RAC2', 'ELMO1', 'WAS']
immune = t_only + b_only + both

# Check if blanks are due to no data 

The follow code chunks show that the following cancers/genes don't have data: colon THBD, Kidney MMP12, and Ovarian MMP12. (As mentioned in EGFR Figure 2 legend)

In [ ]:
#Get append version of the df with all proteins 
df_all_prot_append = pd.read_csv("Pval_corr_table_Fig_2_return_all_append.csv")
df_all_prot_append = df_all_prot_append.drop(['Unnamed: 0'], axis=1)
#Remove Proteomics labeling 
df_all_prot_append = df_all_prot_append.replace(to_replace ='_proteomics', value = '', regex = True) 



In [ ]:
#MHC class2 (CD4) molecules
mhc2 = ["HLA-DPA1","HLA-DPB1","HLA-DMA","HLA-DMB","HLA-DOA","HLA-DOB","HLA-DQ","HLA-DRA","HLA-DRB1"]
#subset dataframe to include genes only desired for figure 
df_all_mhc2 = df_all_prot_append[df_all_prot_append.Comparison.isin(mhc2)]
print("Number of missing columns" )
print(56 - len(df_all_mhc2))
#HLA-DP and HLA-DM missing for all. HLA-DRA and HLA-DRB1 consistently not sig 
df_all_mhc2

In [ ]:
#CD8 and MHC class 1 molecules
mhc1 = ["CD8A","HLA-A","HLA-B","HLA-C"]

df_all_mhc1 = df_all_prot_append[df_all_prot_append.Comparison.isin(mhc1)]
df_all_mhc1

In [ ]:
#Remove Proteomics labeling 
df_all_prot_append = df_all_prot_append.replace(to_replace ='_proteomics', value = '', regex = True) 

#subset dataframe to include genes only desired for figure 
df_all_comp_coag = df_all_prot_append[df_all_prot_append.Comparison.isin(coag_upa)]
print("Number of rows in data frame " + str(len(df_all_comp_coag)))

Our Figure includes 26 genes for 8 cancers. If all data was present there would be 208 rows. However, the data frame only has 205 rows. 3 genes are missing. 

In [ ]:
def find_missing_genes(test_list, full_list):
    for gene in full_list:
        if (gene not in test_list):
            print(gene)
        

In [ ]:
#Get list of genes for colon, kidney, and ovarian
colon = df_all_prot_append[df_all_prot_append["Cancer Type"] == "Colon"]
colon_list = colon.Comparison.to_list()

Kidney = df_all_prot_append[df_all_prot_append["Cancer Type"] == "Kidney"]
Kidney_list = Kidney.Comparison.to_list()

Ovarian = df_all_prot_append[df_all_prot_append["Cancer Type"] == "Ovarian"]
Ovarian_list = Ovarian.Comparison.to_list()

In [ ]:
#Show the 3 missing genes 
print("Ovarian missing genes: ")
find_missing_genes(Ovarian_list, coag_upa)
print("Kidney missing genes: ")
find_missing_genes(Kidney_list, coag_upa)
print("Colon missing genes: ")
find_missing_genes(colon_list, coag_upa)